<hr>
# Data Cleaning
Notebook for initial processing and cleaning of data, prior to EDA.
<hr>
## Steps
<hr>
### 1. Load Shard

In [1]:
# setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [25]:
# load shard
skipRows = [47, 2330]
shard = pd.read_csv('pbp/data/pbp-2013.csv', nrows=10000, skiprows=skipRows)

In [12]:
# check
shard.head()

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,IsTwoPointConversion,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards
0,2013090901,2013-09-09,1,0,0,NaN,HOU,0,0,0,...,0,0,NaN,0,OWN,0,NaN,0,NaN,0
1,2013092906,2013-09-29,2,0,0,NaN,PIT,0,0,0,...,0,0,NaN,0,OWN,0,NaN,0,NaN,0
2,2013111009,2013-11-10,3,0,0,NaN,HOU,0,0,0,...,0,0,NaN,0,OWN,0,NaN,0,NaN,0
3,2013091502,2013-09-15,3,0,0,NaN,CAR,0,0,0,...,0,0,NaN,0,OWN,0,NaN,0,NaN,0
4,2013091502,2013-09-15,3,0,0,NaN,CAR,0,0,0,...,0,0,NaN,0,OWN,0,NaN,0,NaN,0


In [14]:
shard.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 45 columns):
GameId                            10000 non-null int64
GameDate                          10000 non-null object
Quarter                           10000 non-null int64
Minute                            10000 non-null int64
Second                            10000 non-null int64
OffenseTeam                       9194 non-null object
DefenseTeam                       10000 non-null object
Down                              10000 non-null int64
ToGo                              10000 non-null int64
YardLine                          10000 non-null int64
Unnamed: 10                       0 non-null float64
SeriesFirstDown                   10000 non-null int64
Unnamed: 12                       0 non-null float64
NextScore                         10000 non-null int64
Description                       10000 non-null object
TeamWin                           10000 non-null int64
Unnamed: 16  

### 2. Clean Data
#### a. Restrict to offensive plays only

In [26]:
shard = shard[(shard.IsRush==1) | (shard.IsPass==1)]

#### b. Remove unnamed columns

In [27]:
shard = shard.drop(["Unnamed: 10", "Unnamed: 12", "Unnamed: 16", "Unnamed: 17"], axis=1)

In [28]:
shard.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6940 entries, 45 to 9997
Data columns (total 41 columns):
GameId                            6940 non-null int64
GameDate                          6940 non-null object
Quarter                           6940 non-null int64
Minute                            6940 non-null int64
Second                            6940 non-null int64
OffenseTeam                       6940 non-null object
DefenseTeam                       6940 non-null object
Down                              6940 non-null int64
ToGo                              6940 non-null int64
YardLine                          6940 non-null int64
SeriesFirstDown                   6940 non-null int64
NextScore                         6940 non-null int64
Description                       6940 non-null object
TeamWin                           6940 non-null int64
SeasonYear                        6940 non-null int64
Yards                             6940 non-null int64
Formation               

#### c. Parse <code>Description</code> for player names, numbers

In [44]:
import re

pattern = re.compile("(?:\s)([0-9]*-.\.[A-Z]*)")

In [46]:
# select test string
print shard.Description[124]

# test pattern
pattern.findall(shard.Description[124])

(10:31) 5-J.FLACCO PASS SHORT LEFT TO 44-V.LEACH PUSHED OB AT BAL 38 FOR 4 YARDS (59-D.TREVATHAN).


['5-J.FLACCO', '44-V.LEACH']

#### d. Add column for <code>Players</code>

In [49]:
shard["Players"] = shard.Description.apply(lambda x: pattern.findall(x))

In [51]:
shard.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6940 entries, 45 to 9997
Data columns (total 42 columns):
GameId                            6940 non-null int64
GameDate                          6940 non-null object
Quarter                           6940 non-null int64
Minute                            6940 non-null int64
Second                            6940 non-null int64
OffenseTeam                       6940 non-null object
DefenseTeam                       6940 non-null object
Down                              6940 non-null int64
ToGo                              6940 non-null int64
YardLine                          6940 non-null int64
SeriesFirstDown                   6940 non-null int64
NextScore                         6940 non-null int64
Description                       6940 non-null object
TeamWin                           6940 non-null int64
SeasonYear                        6940 non-null int64
Yards                             6940 non-null int64
Formation               

#### e. Add columns for <code>Passer, Rusher, Receiver</code>

In [85]:
# add Passer
shard["Passer"] = shard.where(shard.IsPass==1).apply(lambda x: x["Players"][0] if type(x["Players"])!=float else None, axis=1)

shard.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6940 entries, 45 to 9997
Data columns (total 43 columns):
GameId                            6940 non-null int64
GameDate                          6940 non-null object
Quarter                           6940 non-null int64
Minute                            6940 non-null int64
Second                            6940 non-null int64
OffenseTeam                       6940 non-null object
DefenseTeam                       6940 non-null object
Down                              6940 non-null int64
ToGo                              6940 non-null int64
YardLine                          6940 non-null int64
SeriesFirstDown                   6940 non-null int64
NextScore                         6940 non-null int64
Description                       6940 non-null object
TeamWin                           6940 non-null int64
SeasonYear                        6940 non-null int64
Yards                             6940 non-null int64
Formation               

In [98]:
shard.Passer[:10]

45       7-C.PONDER
74             None
82             None
111      5-J.FLACCO
112            None
113      5-J.FLACCO
115    18-P.MANNING
116            None
117    18-P.MANNING
118    18-P.MANNING
Name: Passer, dtype: object

In [90]:
# add Receiver
shard["Receiver"] = shard.where(shard.IsPass==1).apply(lambda x: x["Players"][1] \
                                                       if (type(x["Players"])!=float) and (len(x["Players"])>1) \
                                                       else None, axis=1)

shard.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6940 entries, 45 to 9997
Data columns (total 44 columns):
GameId                            6940 non-null int64
GameDate                          6940 non-null object
Quarter                           6940 non-null int64
Minute                            6940 non-null int64
Second                            6940 non-null int64
OffenseTeam                       6940 non-null object
DefenseTeam                       6940 non-null object
Down                              6940 non-null int64
ToGo                              6940 non-null int64
YardLine                          6940 non-null int64
SeriesFirstDown                   6940 non-null int64
NextScore                         6940 non-null int64
Description                       6940 non-null object
TeamWin                           6940 non-null int64
SeasonYear                        6940 non-null int64
Yards                             6940 non-null int64
Formation               

In [97]:
shard.Receiver[:10]

45             None
74             None
82             None
111       27-R.RICE
112            None
113    80-B.STOKLEY
115     80-J.THOMAS
116            None
117     83-W.WELKER
118     87-E.DECKER
Name: Receiver, dtype: object

In [93]:
# add Rusher
shard["Rusher"] = shard.where(shard.IsRush==1).apply(lambda x: x["Players"][0] if type(x["Players"])!=float else None, axis=1)\

shard.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6940 entries, 45 to 9997
Data columns (total 45 columns):
GameId                            6940 non-null int64
GameDate                          6940 non-null object
Quarter                           6940 non-null int64
Minute                            6940 non-null int64
Second                            6940 non-null int64
OffenseTeam                       6940 non-null object
DefenseTeam                       6940 non-null object
Down                              6940 non-null int64
ToGo                              6940 non-null int64
YardLine                          6940 non-null int64
SeriesFirstDown                   6940 non-null int64
NextScore                         6940 non-null int64
Description                       6940 non-null object
TeamWin                           6940 non-null int64
SeasonYear                        6940 non-null int64
Yards                             6940 non-null int64
Formation               

In [95]:
shard.Rusher[:10]

45            None
74     38-B.BOLDEN
82     46-A.MORRIS
111           None
112      27-R.RICE
113           None
115           None
116    27-K.MORENO
117           None
118           None
Name: Rusher, dtype: object

### 3. Engineer Features

The purpose of using PBP data is to identify metrics such as Yards Per Carry (YPC) that might be useful for predicting a player's fantasy points.

Metrics of interest:
- Yards Per Carry (Outliers Removed)
- Yards Per Reception
- Yards Per Reception (Outliers Removed)
- Yards Per Carry Allowed
- Yards Per Carry Allowed (Outliers Removed)
- Yards Per Reception Allowed
- Yards Per Reception Allowed (Outliers Removed)
- First Downs Gained Per Game (Relative to league avg)
- Yards Per Play By Team, Formation & Play Type
- % Carries 10+ Yards
- % Receptions 10+ Yards
- Expected Points Added (EPA)
- Adjusted Line Yards (YPC adjusted for defenses, line position, etc. RBs only)
- Distribution of Yards Gained Per Play (for RBs and WRs)

In [55]:
shard.Formation.unique()

array(['UNDER CENTER', 'SHOTGUN', 'NO HUDDLE SHOTGUN', 'NO HUDDLE'], dtype=object)

### Map Player Names to Positions
#### a. Scrape player name, team, number

In [137]:
import bs4
import requests
from string import maketrans

# url to scrape player data, use url.format(year, page)
url = "http://www.foxsports.com/nfl/players?teamId=0&season={}&position=0&page={}&country=0&grouping=0&weightclass=0"

In [283]:
def extract_page_content(content):
    # parse html
    soup = bs4.BeautifulSoup(content, 'html.parser')
    rows = test_soup.find('div', class_=re.compile('wisbb_playersTable')).find("tbody").findAll("tr")
    
    # get raw strings for content of each td
    num_pos = [[el.text.strip() for el in row.findAll("td")[2:4]] for row in rows]
    raw_name = [[el.find("span").text.strip() for el in row.findAll("td")[:1]] for row in rows]
    team = [[el.text.strip() for el in row.findAll("td")[1:2]] for row in rows]
    
    # rearrange raw_name
    name = [[' '.join(el[0].split(",")[::-1]).strip()] for el in raw_name]
    
    return [el[0]+el[1]+el[2] for el in zip(name, team, num_pos)]

def scrape_nfl_roster(year, pages):
    players = []
    for page in xrange(1,pages):
        response = requests.get(url.format(year, page))
        players.append(extract_page_content(response.content))
        
    return players

In [111]:
# create parsing code
test_response = requests.get(url.format(2013, 1))

In [284]:
test_content = extract_page_content(test_response.content)

In [285]:
test_content[:10]

[[u"J.J. 'Unga", u'BUF', u'64', u'T'],
 [u'Isaako Aaitui', u'MIA', u'97', u'DT'],
 [u'Isa Abdul-Quddus', u'DET', u'-', u'S'],
 [u'Husain Abdullah', u'KC', u'39', u'S'],
 [u'Oday Aboushi', u'NYJ', u'75', u'T'],
 [u'John Abraham', u'ARZ', u'55', u'LB'],
 [u'Emmanuel Acho', u'PHI', u'51', u'LB'],
 [u'Sam Acho', u'ARZ', u'94', u'LB'],
 [u'Aaron Adams', u'GB', u'77', u'T'],
 [u'Corey Adams', u'ATL', u'48', u'LB']]

#### b. Create Postgres table with player info

In [227]:
import psycopg2

In [238]:
# open connetion to db
try:
    conn = psycopg2.connect(database="fantasyanalytics")
except:
    print "ERROR: Unable to connect to db."

In [240]:
# setup cursor
cur = conn.cursor()

In [295]:
# create new table
cur.execute("CREATE TABLE players (id serial PRIMARY KEY, name text, team varchar(3), number varchar(2), position varchar(3));")

In [296]:
# insert test rows into table
test_string = ",".join(cur.mogrify("(%s,%s,%s,%s)", x) for x in test_content)
cur.execute("INSERT INTO players (name, team, number, position) VALUES " + test_string)

In [298]:
# check rows were inserted correctly
cur.execute("SELECT * FROM players LIMIT 10")
for record in cur:
    print record

(1, "J.J. 'Unga", 'BUF', '64', 'T')
(2, 'Isaako Aaitui', 'MIA', '97', 'DT')
(3, 'Isa Abdul-Quddus', 'DET', '-', 'S')
(4, 'Husain Abdullah', 'KC', '39', 'S')
(5, 'Oday Aboushi', 'NYJ', '75', 'T')
(6, 'John Abraham', 'ARZ', '55', 'LB')
(7, 'Emmanuel Acho', 'PHI', '51', 'LB')
(8, 'Sam Acho', 'ARZ', '94', 'LB')
(9, 'Aaron Adams', 'GB', '77', 'T')
(10, 'Corey Adams', 'ATL', '48', 'LB')


#### c. Map player id from <code>players</code> table to <code>Passer, Rusher, Receiver</code> in play data

In [302]:
shard[["OffenseTeam", "Players", "Passer", "Rusher", "Receiver"]].head()

,OffenseTeam,Players,Passer,Rusher,Receiver
45,MIN,[7-C.PONDER],7-C.PONDER,None,None
74,NE,[38-B.BOLDEN],None,38-B.BOLDEN,None
82,WAS,[46-A.MORRIS],None,46-A.MORRIS,None
111,BAL,"[5-J.FLACCO, 27-R.RICE]",5-J.FLACCO,None,27-R.RICE
112,BAL,[27-R.RICE],None,27-R.RICE,None
